# Notebook Taxa Selic e IPCA (2014 A 2024) 

Verificar a eficácia da Política monetária no combate a inflação

Abaixo um roteiro em células de notebook que cobre:

1. Importação de pacotes
2. Carregar os dados JSON
3. Unir datasets (merge por data)
4. Criar retorno mensal do IBOV (%)
5. Estatísticas descritivas
6. Gráficos comparativos
7. Regressão Linear Simples (modelo principal)
8. Gráfico de dispersão com linha de regressão
9. Análise complementar: SELIC Meta (% a.a.)


## 1. Importação de pacotes

In [11]:
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

## 2. Carregar os dados Selic x IPCA

In [29]:
base_path = Path("../dados")
file = base_path / "history_selic_ipca_2014__2024.csv"

# 1) Carregar
df = pd.read_csv(
    file, sep=";", decimal=",", parse_dates=["date_reference"], dayfirst=True
)

# 2) Criar defasagens de 1..12 da SELIC
for k in range(1, 13):
    df[f"selic_l{k}"] = df["selic"].shift(k)

# (opcional) trabalhar com ΔSELIC para estacionaridade
# df["dselic"] = df["selic"].diff()
# for k in range(1, 13):
#     df[f"dselic_l{k}"] = df["dselic"].shift(k)

# 3) Escolha: em nível OU em diferenças (eu recomendo ΔSELIC)
# lags_delta = [f"dselic_l{k}" for k in range(1,13)]
lags_meta = [f"selic_l{k}" for k in range(1,13)]
# lags = [f"selic_l{k}" for k in range(1,13)]  # se quiser nível


In [19]:
lags_meta

['selic_l1',
 'selic_l2',
 'selic_l3',
 'selic_l4',
 'selic_l5',
 'selic_l6',
 'selic_l7',
 'selic_l8',
 'selic_l9',
 'selic_l10',
 'selic_l11',
 'selic_l12']

In [20]:
X = df[lags_meta].dropna()
y = df.loc[X.index, "ipca"]

Xc = sm.add_constant(X)
ols = sm.OLS(y, Xc).fit(cov_type="HAC", cov_kwds={"maxlags":12})  # HAC/Newey-West
print(ols.summary())

# 4) Efeito cumulativo (soma dos betas das defasagens)
beta_sum = ols.params[[c for c in Xc.columns if c.startswith("dselic_l") or c.startswith("selic_l")]].sum()
print("Efeito cumulativo dos 12 lags:", beta_sum)

# 5) VIF (checar multicolinearidade entre lags)
vif = pd.Series([variance_inflation_factor(X.values, i) for i in range(X.shape[1])],
                index=X.columns)
print(vif.sort_values(ascending=False))


                            OLS Regression Results                            
Dep. Variable:                   ipca   R-squared:                       0.203
Model:                            OLS   Adj. R-squared:                  0.113
Method:                 Least Squares   F-statistic:                     13.49
Date:                Sun, 19 Oct 2025   Prob (F-statistic):           1.57e-16
Time:                        13:45:20   Log-Likelihood:                -48.264
No. Observations:                 120   AIC:                             122.5
Df Residuals:                     107   BIC:                             158.8
Df Model:                          12                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.4621      0.137      3.364      0.0

In [31]:
df.dropna()

,date_reference,ipca,selic,selic_l1,selic_l2,selic_l3,selic_l4,selic_l5,selic_l6,selic_l7,selic_l8,selic_l9,selic_l10,selic_l11,selic_l12
12,2015-01-01,1.24,12.25,11.75,11.25,11.25,11.00,11.00,11.00,11.00,11.00,11.00,10.75,10.75,10.50
13,2015-02-01,1.22,12.25,12.25,11.75,11.25,11.25,11.00,11.00,11.00,11.00,11.00,11.00,10.75,10.75
14,2015-03-01,1.32,12.75,12.25,12.25,11.75,11.25,11.25,11.00,11.00,11.00,11.00,11.00,11.00,10.75
15,2015-04-01,0.71,13.25,12.75,12.25,12.25,11.75,11.25,11.25,11.00,11.00,11.00,11.00,11.00,11.00
16,2015-05-01,0.74,13.25,13.25,12.75,12.25,12.25,11.75,11.25,11.25,11.00,11.00,11.00,11.00,11.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,2024-08-01,-0.02,10.50,10.50,10.50,10.50,10.75,10.75,11.25,11.75,11.75,12.25,12.75,12.75,13.25
128,2024-09-01,0.44,10.75,10.50,10.50,10.50,10.50,10.75,10.75,11.25,11.75,11.75,12.25,12.75,12.75
129,2024-10-01,0.56,10.75,10.75,10.50,10.50,10.50,10.50,10.75,10.75,11.25,11.75,11.75,12.25,12.75
130,2024-11-01,0.39,11.25,10.75,10.75,10.50,10.50,10.50,10.50,10.75,10.75,11.25,11.75,11.75,12.25
